In [1]:
import datetime
import pandas as pd
from influxdb import DataFrameClient
import matplotlib
import matplotlib.pyplot as plt
import json
import pytz
import numpy as np
import yaml
%matplotlib inline

In [6]:
with open('config.yaml') as fp:
    config = yaml.safe_load(fp)
database_config = config.get('database')

In [3]:
with open('uuid_map.json') as fp:
    uuid_map = json.load(fp)

In [7]:
client = DataFrameClient(
    host=database_config.get('host'), 
    port=database_config.get('port'), 
    username=database_config.get('username'), 
    password=database_config.get('password'), 
    database=database_config.get('database'), 
    ssl=database_config.get('ssl'), 
    verify_ssl=database_config.get('verify_ssl')
)

In [8]:
tz_local = pytz.timezone('America/Los_Angeles')
tz_utc = pytz.timezone('UTC')

In [9]:
uuid_dict = {
    'building_power': uuid_map['xbos/wattnode/building_main/PowerSum'],
    'freezer_power': uuid_map['xbos/wattnode/fre_comp_evapfan/PowerSum'],
    'ref_comp_power': uuid_map['xbos/wattnode/ref_comp/PowerSum'],
    'ref_fan_power': uuid_map['xbos/wattnode/ref_evapfan/PowerSum'],
    'hvac_west_power': uuid_map['xbos/wattnode/hvac_west_comp/PowerSum'],
    'hvac_east_power': uuid_map['xbos/wattnode/hvac_east_comp/PowerSum'],
    
    'east_temperature': uuid_map['xbos/flexstat/thermostat_east/space_temp'],
    'west_temperature': uuid_map['xbos/flexstat/thermostat_west/space_temp'],
    'freezer_temperature': uuid_map['xbos/parker/freezer/CabinetTemperature'],
    'ref_temperature': uuid_map['xbos/parker/refrigerator/CabinetTemperature'],
    
    'east_heating_sp': uuid_map['xbos/flexstat/thermostat_east/active_heating_setpt'],
    'east_cooling_sp': uuid_map['xbos/flexstat/thermostat_east/active_cooling_setpt'],
    'west_heating_sp': uuid_map['xbos/flexstat/thermostat_west/active_heating_setpt'],
    'west_cooling_sp': uuid_map['xbos/flexstat/thermostat_west/active_cooling_setpt'],
    'freezer_sp': uuid_map['xbos/parker/freezer/Setpoint'],
    'ref_sp': uuid_map['xbos/parker/refrigerator/Setpoint'],
    
    'soc_battery': '0efc4fa5-755c-5c45-863a-c0c776ab7538',
    'setpoint_battery': '276ea28f-0f74-5b3e-9ad1-f3b9f747dbe4',
    'pv_generation_battery': 'fdfd7bbb-d2da-5b11-a8fa-58b231ab9802',
    
    'oat_current': 'f7c1f2c8-c996-528c-ab3d-bdc96dc9cf72',
    'humidity_current': '7967b372-2699-57e1-bc15-7861bfe6d024',
    'windspeed_current': 'ac555599-0403-5bea-8441-f39a4e8e0dac',
    'cloudcover_current': '6cce3e9a-3822-551b-b13c-f4b874f3afa1',
    'solar_current': 'd15979be-7a63-5230-9e7b-d068d9f40b08',    
    
    'oat_forecast': '69be4db0-48f5-592f-b5a1-e2e695f28ad1',
    'humidity_forecast': 'c2379487-3df3-5bfe-bd8d-5992d2381ed5',
    'windspeed_forecast': '837d0588-b30a-56ee-920f-cf366fa0871f',
    'cloudcover_forecast': '6f93857f-50a6-5b9f-8b94-0231f511b382',
    'solar_forecast': 'a8357adb-c59d-5316-a0e8-51d2b2948c75',
    
    'price_energy_dr': '3be4c234-a38a-5e73-9d53-4503751592be',
    'price_demand_dr': '90928e8d-df40-5e75-9ddb-7ee444bc187f',
    'pmax_dr': '522605a9-77b1-57e3-9fac-06dd83ab8e89',
    'pmin_dr': '6b42adf8-3a48-5ae7-bdc3-19226e602865',
    
    'pv_generation_rtac': 'd4102860-1f55-5e14-b02d-8bdb9e987676',
    'soc_rtac': '944a5aba-8c59-586e-af87-d95151596cf1'
}

In [10]:
def parse_dt_utc(dt):
    return tz_local.localize(datetime.datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")).astimezone(tz_utc)

In [11]:
def current_uuid_data(client, uuid, st, et, column_name):
    st_utc = parse_dt_utc(st).strftime("%Y-%m-%dT%H:%M:%SZ")
    et_utc = parse_dt_utc(et).strftime("%Y-%m-%dT%H:%M:%SZ")
    query = "select value from timeseries where \"uuid\" = '%s' and time>= '%s' and time <= '%s'" % (uuid, st_utc, et_utc)
    df = client.query(query)['timeseries']
    df = df[['value']]
    df.columns = [column_name]
    return df

In [12]:
def forecast_uuid_data(client, uuid, st, et, column_name):
    st_utc = parse_dt_utc(st).strftime("%Y-%m-%dT%H:%M:%SZ")
    et_utc = parse_dt_utc(et).strftime("%Y-%m-%dT%H:%M:%SZ")
    query = "select value from timeseries where \"uuid\" = '%s' and \"prediction_step\" = '1' and time>= '%s' and time <= '%s'" % (uuid, st_utc, et_utc)
    df = client.query(query)['timeseries']
    df = df[['value']]
    df.columns = [column_name]
    return df

In [13]:
def get_data_section(client, endswith, st, et, uuid_dict, resample='1T', current=True):
    df_list = []
    for variable in uuid_dict:
        if variable.endswith(endswith):
            uuid = uuid_dict[variable]
            if current:
                df = current_uuid_data(client, uuid, st, et, column_name=variable[:variable.rfind('_')])
            else:
                df = forecast_uuid_data(client, uuid, st, et, column_name=variable[:variable.rfind('_')])
            df = df.resample(resample).mean()
            df_list.append(df)
    return pd.concat(df_list, axis=1)

In [24]:
st = "2020-08-24 00:00:00"
et = "2020-08-25 23:59:59"

In [25]:
power_df = get_data_section(client, '_power', st, et, uuid_dict, '1T')
power_df = power_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
power_df.head()

,building,freezer,ref_comp,ref_fan,hvac_west,hvac_east
2020-08-24 00:00:00,33669.442383,3953.234660,1340.061724,283.620580,1424.373414,2642.562684
2020-08-24 00:30:00,32682.601855,2885.960037,1323.542357,300.780708,359.681568,3860.252844
2020-08-24 01:00:00,32734.888444,2647.788108,1136.811967,303.672582,358.381685,3839.434924
2020-08-24 01:30:00,32298.399674,3001.012548,1305.671146,275.385274,357.908542,3862.473116
2020-08-24 02:00:00,32056.195703,2928.162652,1142.082186,264.540468,358.202979,3834.648039


In [26]:
setpoint_df = get_data_section(client, '_sp', st, et, uuid_dict, '1T')
setpoint_df = setpoint_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
setpoint_df.head()

,east_heating,east_cooling,west_heating,west_cooling,freezer,ref
2020-08-24 00:00:00,68.0,70.0,68.0,70.0,-7,33
2020-08-24 00:30:00,68.0,70.0,68.0,70.0,-7,33
2020-08-24 01:00:00,68.0,70.0,68.0,70.0,-7,33
2020-08-24 01:30:00,68.0,70.0,68.0,70.0,-7,33
2020-08-24 02:00:00,68.0,70.0,68.0,70.0,-7,33


In [27]:
temperature_df = get_data_section(client, '_temperature', st, et, uuid_dict, '1T')
temperature_df = temperature_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
temperature_df.head()

,east,west,freezer,ref
2020-08-24 00:00:00,70.653998,70.082999,-3.133333,34.300000
2020-08-24 00:30:00,71.822998,70.152998,-5.533333,34.333333
2020-08-24 01:00:00,71.914999,70.180332,-5.333333,34.266667
2020-08-24 01:30:00,71.924999,70.224998,-5.166667,34.166667
2020-08-24 02:00:00,71.791665,70.118666,-5.400000,34.200000


In [28]:
battery_df = get_data_section(client, '_battery', st, et, uuid_dict, '1T')
battery_df = battery_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
battery_df.head()

,soc,setpoint,pv_generation
2020-08-24 00:00:00,0.454153,0,0.0
2020-08-24 00:30:00,0.454153,0,0.0
2020-08-24 01:00:00,0.454153,0,0.0
2020-08-24 01:30:00,0.454153,0,0.0
2020-08-24 02:00:00,0.454153,0,0.0


In [29]:
weather_df = get_data_section(client, '_current', st, et, uuid_dict, '1T')
weather_df = weather_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
weather_df.head()

,oat,humidity,windspeed,cloudcover,solar
2020-08-24 00:00:00,55.894368,84.000000,1.966575,0.506000,0.0
2020-08-24 00:30:00,55.341978,84.700000,1.918067,0.596000,0.0
2020-08-24 01:00:00,55.089200,84.797778,1.944956,0.685956,0.0
2020-08-24 01:30:00,55.423689,82.602222,2.083978,0.775022,0.0
2020-08-24 02:00:00,55.833200,80.397778,2.267844,0.865978,0.0


In [30]:
rtac_df = get_data_section(client, '_rtac', st, et, uuid_dict, '1T')
rtac_df = rtac_df.tz_convert(tz_local).tz_localize(None).resample('1T').mean().interpolate('linear').resample('30T').mean()
rtac_df.head()

,pv_generation,soc
2020-08-24 00:00:00,0.0,0.908611
2020-08-24 00:30:00,0.0,0.939946
2020-08-24 01:00:00,0.0,0.657878
2020-08-24 01:30:00,0.0,0.939865
2020-08-24 02:00:00,0.0,0.939907


In [31]:
dr_df = get_data_section(client, '_dr', st, et, uuid_dict, '60T', current=False)
dr_df = dr_df.tz_convert(tz_local).tz_localize(None).resample('1T').fillna(method='ffill')
dr_df.head()

,price_energy,price_demand,pmax,pmin
2020-08-24 00:00:00,0.04026,0.0,999999,-999999
2020-08-24 00:01:00,0.04026,0.0,999999,-999999
2020-08-24 00:02:00,0.04026,0.0,999999,-999999
2020-08-24 00:03:00,0.04026,0.0,999999,-999999
2020-08-24 00:04:00,0.04026,0.0,999999,-999999
